In [20]:
from google.cloud import bigquery
import os
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import datetime
from google.cloud import bigquery
import pytz
from google.oauth2 import service_account
import numpy as np
import requests
from datetime import datetime as dt
from datetime import date


In [21]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'.\creds\opendatadbt-f2e674527eb6.json'
client = bigquery.Client()

credentials = ServiceAccountCredentials.from_json_keyfile_name(r'.\creds\opendatadbt-f2e674527eb6.json') # Your json file here



chattanooga_table_id = "opendatadbt.311.chattanooga311"

#Inital Load
chattanooga311 = requests.get("https://www.chattadata.org/resource/8qb9-5fja.json?$limit=10000&$where=created_date >= '2021-03-22'")
chattanooga311json = chattanooga311.json()
chattanooga_df = pd.json_normalize(chattanooga311json)
chattanooga_df_parsed = chattanooga_df['publiclocation.coordinates'].apply(pd.Series)
chattanooga_df_cleaned = pd.concat([chattanooga_df.drop(['publiclocation.coordinates'], axis=1), chattanooga_df_parsed], axis=1)
mapping = {chattanooga_df_cleaned.columns[-1]:'latitude', chattanooga_df_cleaned.columns[-2]:'longitude',chattanooga_df_cleaned.columns[-3]:'publiclocationtype'}
chattanooga_df_cleaned = chattanooga_df_cleaned.rename(columns=mapping)
chattanooga_df_cleaned = chattanooga_df_cleaned.drop(['publiclocation.type'], axis=1)
cols = ['service_request_key','created_date','completed_at','department','request_type','status_code','latitude','longitude']
chattanooga_df_cleaned = pd.DataFrame(chattanooga_df_cleaned,columns = cols)
chattanooga_df_cleaned

In [22]:
#Incremental load
chattanooga_table_id = "opendatadbt.311.chattanooga311"
current_datetime = dt.today().strftime("%Y-%m-%dT%H:%M:%S.000")
query_string = """
SELECT 
max(created_date), max(completed_at)
FROM `opendatadbt.311.chattanooga311`
limit 10
"""

chattanoogamaxopendatedataframe = (
    client.query(query_string)
    .result()
    .to_dataframe()
)
chattanoogamaxopendatedate = str(chattanoogamaxopendatedataframe['f0_'][0])

chattanoogamaxclosedatedataframe = (
    client.query(query_string)
    .result()
    .to_dataframe()
)
chattanoogamaxclosedate = str(chattanoogamaxclosedatedataframe['f1_'][0])



In [23]:
#Incremental Load

chattanooga311 = requests.get("https://www.chattadata.org/resource/8qb9-5fja.json?$limit=50000&$where=created_date>'"+chattanoogamaxopendatedate+"' OR completed_at between '" +chattanoogamaxclosedate+ "' AND '" + current_datetime + "'")
chattanooga311json = chattanooga311.json()
chattanooga_df = pd.json_normalize(chattanooga311json)
chattanooga_df_parsed = chattanooga_df['publiclocation.coordinates'].apply(pd.Series)
chattanooga_df_cleaned = pd.concat([chattanooga_df.drop(['publiclocation.coordinates'], axis=1), chattanooga_df_parsed], axis=1)
mapping = {chattanooga_df_cleaned.columns[-1]:'latitude', chattanooga_df_cleaned.columns[-2]:'longitude',chattanooga_df_cleaned.columns[-3]:'publiclocationtype'}
chattanooga_df_cleaned = chattanooga_df_cleaned.rename(columns=mapping)
chattanooga_df_cleaned = chattanooga_df_cleaned.drop(['publiclocation.type'], axis=1)
cols = ['service_request_key','created_date','completed_at','department','request_type','status_code','latitude','longitude']
chattanooga_df_cleaned = pd.DataFrame(chattanooga_df_cleaned,columns = cols)
chattanooga_df_cleaned

,service_request_key,created_date,completed_at,department,request_type,status_code,latitude,longitude
0,9768359,2021-03-24T22:55:02.000,NaN,NaN,Bulk Trash,O-NEW,35.132115,-85.194581
1,9766588,2021-03-24T16:16:53.000,NaN,NaN,Missed Garbage,O-NEW,35.036170,-85.261417
2,9766222,2021-03-24T15:31:11.000,NaN,NaN,Bulk Trash,O-NEW,35.035778,-85.259986
3,9766305,2021-03-24T15:41:30.000,NaN,NaN,Bulk Trash,O-NEW,35.066682,-85.296048
4,9767089,2021-03-24T17:16:57.000,NaN,NaN,Bulk Trash,O-NEW,35.111921,-85.275704
...,...,...,...,...,...,...,...,...
125,9768449,2021-03-24T23:50:07.000,NaN,NaN,Bulk Trash,O-NEW,NaN,NaN
126,9767447,2021-03-24T18:16:33.000,NaN,NaN,Brush Collection,O-NEW,NaN,NaN
127,9767233,2021-03-24T17:39:25.000,NaN,NaN,Bagged Yard Waste,O-NEW,NaN,NaN
128,9766410,2021-03-24T15:53:15.000,NaN,NaN,Abandoned or Inoperable Vehicle (Private Prope...,O-NEW,NaN,NaN


In [24]:
chattanooga_table_id = "opendatadbt.311.chattanooga311"
dataset_ref = client.dataset('311')
table_ref = dataset_ref.table('chattanooga311')
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",
)

client.load_table_from_dataframe(
    chattanooga_df_cleaned, table_ref, job_config=job_config
).result()  # Make an API request.
